In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .master("local[*]")
        .appName("UDF")
        .getOrCreate()
)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/spark-3.4.1-bin-hadoop3/jars/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/spark-3.4.1-bin-hadoop3/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
log4j:WARN No appenders could be found for logger (org.apache.spark.util.ShutdownHookManager).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


In [2]:
train = spark.read.csv("../data/train.csv", inferSchema=True, header=True).cache()

In [3]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col
from pyspark.sql.types import LongType

def to_months(ms):
    return ms // 31536000000 // 12

to_months_udf = f.udf(to_months, LongType())

In [6]:
%%time
(
    train
        .select("content_id", to_months_udf("timestamp").alias("month"))
        .groupBy("content_id")
        .mean("month")
        .show()
)

+----------+----------+
|content_id|avg(month)|
+----------+----------+
|      4519|       0.0|
|      4818|       0.0|
|      5518|       0.0|
|     13285|       0.0|
|     12027|       0.0|
|       833|       0.0|
|      9427|       0.0|
|       496|       0.0|
|      5156|       0.0|
|      6336|       0.0|
|      2866|       0.0|
|      1959|       0.0|
|      7982|       0.0|
|     23336|       0.0|
|       148|       0.0|
|      1342|       0.0|
|      1088|       0.0|
|      1580|       0.0|
|       471|       0.0|
|      2122|       0.0|
+----------+----------+
only showing top 20 rows

CPU times: user 67.3 ms, sys: 49.9 ms, total: 117 ms
Wall time: 3min 11s


In [5]:
%%time
(
 train
    .select("content_id", (col("timestamp") / 31536000000 / 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show()
)

+----------+--------------------+
|content_id|          avg(month)|
+----------+--------------------+
|      4519|0.015429677931821352|
|      4818|0.016230649272587192|
|      5518| 0.02103952664360606|
|     13285| 0.04141825984987046|
|     12027| 0.04150963056316861|
|       833|0.022584471034752644|
|      9427| 0.02158891106034129|
|       496|0.020155351740181845|
|      5156|0.018064186032779997|
|      6336|0.019195051162991618|
|      2866|0.026921431007821156|
|      1959| 0.02670915106444127|
|      7982| 0.02122819619071833|
|     23336|0.019260306662998706|
|       148|0.017910791660268703|
|      1342| 0.02195320830526077|
|      1088| 0.02461949633696784|
|      1580|0.022461073541947943|
|       471| 0.02611517946736722|
|      2122|0.026057020501248997|
+----------+--------------------+
only showing top 20 rows

CPU times: user 25.7 ms, sys: 13.5 ms, total: 39.2 ms
Wall time: 12.9 s
